# NYC Taxi Trip Data Analysis - Modeling

This notebook covers the model training, evaluation, and interpretation steps for the NYC Taxi Trip Data Analysis project.

## 1. Setup and Imports

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime

# Add the project root directory to the Python path
sys.path.append('..')

# Import project modules
from src.config import RESULTS_DIR, MODELS_DIR
from src.data.feature_selection import (
    select_features_mutual_info, select_features_lasso,
    compare_feature_selection_methods, get_common_features
)
from src.models.trainer import (
    prepare_data_for_modeling, train_baseline_model, train_linear_regression, 
    train_ridge_regression, train_random_forest, train_gradient_boosting,
    train_knn_regressor, train_svm_regressor, train_naive_bayes,
    evaluate_model, get_feature_importance
)
from src.models.evaluator import (
    plot_residuals, plot_prediction_vs_actual, plot_error_distribution,
    plot_feature_importance, compare_models
)
from src.utils.helpers import save_metrics, timer_decorator

# Set up plotting
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Create directories if they don't exist
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

## 2. Load Processed Data

Let's load the processed data from the preprocessing notebook.

In [ ]:
# Load the processed data
processed_dir = os.path.join(RESULTS_DIR, 'processed_data')
taxi_type = 'yellow'

# Check if processed data exists
processed_file = os.path.join(processed_dir, f"{taxi_type}_features.csv")
if os.path.exists(processed_file):
    df = pd.read_csv(processed_file)
    print(f"Loaded processed data from {processed_file}")
else:
    print(f"Processed data file {processed_file} not found. Please run the preprocessing notebook first.")
    # If the file doesn't exist, we'll use the data from the main script output
    # This is just a fallback in case the preprocessing notebook hasn't been run
    from src.data.loader import load_taxi_data
    from src.data.cleaner import clean_yellow_taxi_data
    from src.data.feature_engineering import engineer_features
    
    print("Loading and processing data directly...")
    raw_df = load_taxi_data(taxi_type, ['2025-01', '2025-02'])
    # Sample for faster processing
    if len(raw_df) > 100000:
        raw_df = raw_df.sample(n=100000, random_state=42)
    clean_df = clean_yellow_taxi_data(raw_df)
    df = engineer_features(clean_df, taxi_type)
    print(f"Processed data shape: {df.shape}")

# Load selected features if available
selected_features_file = os.path.join(processed_dir, f"{taxi_type}_selected_features.txt")
if os.path.exists(selected_features_file):
    with open(selected_features_file, 'r') as f:
        selected_features = [line.strip() for line in f.readlines()]
    print(f"Loaded {len(selected_features)} selected features from {selected_features_file}")
else:
    selected_features = None
    print("No selected features file found. Will select features during modeling.")

## 3. Prepare Data for Modeling

Let's prepare the data for modeling by splitting it into training and testing sets.

In [ ]:
# Prepare data for modeling
target_col = 'trip_duration'
if target_col not in df.columns:
    raise ValueError(f"Target column '{target_col}' not found in data.")

# Prepare data
X_train, X_test, y_train, y_test, cat_cols, num_cols = prepare_data_for_modeling(df, target_col)
print(f"Data split into train ({len(X_train)} rows) and test ({len(X_test)} rows) sets")
print(f"Categorical columns ({len(cat_cols)}): {cat_cols}")
print(f"Numerical columns ({len(num_cols)}): {num_cols}")

## 4. Feature Selection

Let's select the most important features for modeling if we haven't already done so in the preprocessing notebook.

In [ ]:
# Apply feature selection if we don't have selected features from preprocessing
if selected_features is None:
    max_features = 20
    print(f"Applying feature selection to reduce features (max: {max_features})")
    
    # Compare different feature selection methods
    feature_selection_results = compare_feature_selection_methods(X_train, y_train, k=max_features)
    
    # Display the features selected by each method
    for method, features in feature_selection_results.items():
        print(f"{method} ({len(features)} features): {features}")
    
    # Get common features selected by at least 2 methods
    selected_features = get_common_features(feature_selection_results, min_methods=2)
    print(f"\nCommon features selected by at least 2 methods ({len(selected_features)}):\n{selected_features}")
    
    # If we have too many features, use mutual information to select top max_features
    if len(selected_features) > max_features:
        print(f"Selected {len(selected_features)} features, reducing to {max_features} using mutual information")
        _, selected_features = select_features_mutual_info(X_train, y_train, k=max_features)
    
    # If we have too few features, use mutual information to select features
    if len(selected_features) < 5:
        print(f"Only {len(selected_features)} features selected, using mutual information to select {max_features}")
        _, selected_features = select_features_mutual_info(X_train, y_train, k=max_features)

# Filter X_train and X_test to include only selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Update categorical and numerical columns
cat_cols_selected = [col for col in cat_cols if col in selected_features]
num_cols_selected = [col for col in num_cols if col in selected_features]

print(f"\nFinal selected features ({len(selected_features)}):\n{selected_features}")
print(f"\nSelected categorical columns ({len(cat_cols_selected)}): {cat_cols_selected}")
print(f"Selected numerical columns ({len(num_cols_selected)}): {num_cols_selected}")

## 5. Train Baseline Model

Let's train a baseline model to establish a performance benchmark.

In [ ]:
# Train a baseline model (mean prediction)
print("Training Baseline model (mean prediction)")
baseline_model, baseline_metrics = train_baseline_model(X_train_selected, y_train, cat_cols_selected, num_cols_selected, strategy='mean')
baseline_test_metrics = evaluate_model(baseline_model, X_test_selected, y_test)
baseline_metrics.update(baseline_test_metrics)

# Display baseline metrics
print("\nBaseline Model Metrics:")
for metric, value in baseline_metrics.items():
    print(f"{metric}: {value}")

# Make predictions
y_pred_baseline = baseline_model.predict(X_test_selected)

# Plot predicted vs. actual values
plot_prediction_vs_actual(y_test, y_pred_baseline, title='Baseline Model: Predicted vs. Actual Trip Duration')

## 6. Train and Evaluate Multiple Models

Let's train and evaluate multiple regression models to predict trip duration.

In [ ]:
# Define a function to train and evaluate a model
@timer_decorator
def train_and_evaluate_model(model_name, X_train, y_train, X_test, y_test, cat_cols, num_cols):
    print(f"Training {model_name} model...")
    
    # Train the model based on the model name
    if model_name == 'Linear Regression':
        model, train_metrics = train_linear_regression(X_train, y_train, cat_cols, num_cols)
    elif model_name == 'Ridge Regression':
        model, train_metrics = train_ridge_regression(X_train, y_train, cat_cols, num_cols)
    elif model_name == 'Random Forest':
        model, train_metrics = train_random_forest(X_train, y_train, cat_cols, num_cols)
    elif model_name == 'Gradient Boosting':
        model, train_metrics = train_gradient_boosting(X_train, y_train, cat_cols, num_cols)
    elif model_name == 'K-Nearest Neighbors':
        model, train_metrics = train_knn_regressor(X_train, y_train, cat_cols, num_cols)
    elif model_name == 'Support Vector Machine':
        model, train_metrics = train_svm_regressor(X_train, y_train, cat_cols, num_cols)
    elif model_name == 'Naive Bayes':
        model, train_metrics = train_naive_bayes(X_train, y_train, cat_cols, num_cols)
    else:
        raise ValueError(f"Unsupported model: {model_name}")
    
    # Evaluate the model on the test set
    test_metrics = evaluate_model(model, X_test, y_test)
    train_metrics.update(test_metrics)
    
    # Get feature importance if available
    if model_name in ['Random Forest', 'Gradient Boosting']:
        importance_df = get_feature_importance(model, cat_cols, num_cols)
    else:
        importance_df = pd.DataFrame({'feature': [], 'importance': []})
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    return model, train_metrics, importance_df, y_pred

In [ ]:
# Define the models to train
models_to_train = [
    'Linear Regression',
    'Ridge Regression',
    'Random Forest',
    'Gradient Boosting',
    'K-Nearest Neighbors',
    'Support Vector Machine',
    'Naive Bayes'
]

# Train and evaluate each model
trained_models = {}
model_metrics = {}
model_importances = {}
model_predictions = {}

# Add baseline model
trained_models['Baseline (Mean)'] = baseline_model
model_metrics['Baseline (Mean)'] = baseline_metrics
model_predictions['Baseline (Mean)'] = y_pred_baseline

# Train and evaluate each model
for model_name in models_to_train:
    model, metrics, importance_df, y_pred = train_and_evaluate_model(
        model_name, X_train_selected, y_train, X_test_selected, y_test, cat_cols_selected, num_cols_selected
    )
    
    trained_models[model_name] = model
    model_metrics[model_name] = metrics
    model_importances[model_name] = importance_df
    model_predictions[model_name] = y_pred

## 7. Compare Model Performance

Let's compare the performance of all the models.

In [ ]:
# Compare models based on test RMSE
compare_models(model_metrics, metric='test_rmse', title='Model Comparison: Test RMSE')

In [ ]:
# Compare models based on test R²
compare_models(model_metrics, metric='test_r2', title='Model Comparison: Test R²')

In [ ]:
# Create a summary table of model metrics
metrics_df = pd.DataFrame({
    'Model': list(model_metrics.keys()),
    'Train RMSE': [metrics['train_rmse'] for metrics in model_metrics.values()],
    'Test RMSE': [metrics['test_rmse'] for metrics in model_metrics.values()],
    'Train R²': [metrics['train_r2'] for metrics in model_metrics.values()],
    'Test R²': [metrics['test_r2'] for metrics in model_metrics.values()]
})

# Sort by test RMSE (ascending)
metrics_df = metrics_df.sort_values('Test RMSE')

# Display the summary table
metrics_df

## 8. Analyze the Best Model

Let's analyze the best-performing model in more detail.

In [ ]:
# Identify the best model based on test RMSE
best_model_name = metrics_df.iloc[0]['Model']
best_model = trained_models[best_model_name]
best_metrics = model_metrics[best_model_name]
best_predictions = model_predictions[best_model_name]

print(f"Best model: {best_model_name}")
print("\nBest model metrics:")
for metric, value in best_metrics.items():
    if isinstance(value, (int, float)):
        print(f"{metric}: {value}")

In [ ]:
# Plot predicted vs. actual values for the best model
plot_prediction_vs_actual(y_test, best_predictions, title=f'{best_model_name}: Predicted vs. Actual Trip Duration')

In [ ]:
# Plot residuals for the best model
plot_residuals(y_test, best_predictions, title=f'{best_model_name}: Residual Analysis')

In [ ]:
# Plot error distribution for the best model
plot_error_distribution(y_test, best_predictions, title=f'{best_model_name}: Error Distribution')

In [ ]:
# Plot feature importance for the best model if available
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    importance_df = model_importances[best_model_name]
    plot_feature_importance(importance_df, title=f'{best_model_name}: Feature Importance', top_n=20)

## 9. Save the Best Model

Let's save the best model for future use.

In [ ]:
# Create a directory for the best model
best_model_dir = os.path.join(MODELS_DIR, taxi_type, 'best_model')
os.makedirs(best_model_dir, exist_ok=True)

# Save the best model
best_model_path = os.path.join(best_model_dir, 'best_model.pkl')
with open(best_model_path, 'wb') as f:
    pickle.dump(best_model, f)
print(f"Saved best model to {best_model_path}")

# Save the best model metrics
best_metrics_path = os.path.join(best_model_dir, 'best_model_metrics.json')
save_metrics(best_metrics, best_metrics_path)
print(f"Saved best model metrics to {best_metrics_path}")

# Save the selected features
selected_features_path = os.path.join(best_model_dir, 'selected_features.txt')
with open(selected_features_path, 'w') as f:
    f.write('\n'.join(selected_features))
print(f"Saved selected features to {selected_features_path}")

## 10. Interpret the Results and Make Recommendations

Let's interpret the results and make recommendations based on our analysis.

In [ ]:
# Analyze feature importance if available
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    importance_df = model_importances[best_model_name]
    top_features = importance_df.head(10)
    
    print(f"Top 10 most important features for {best_model_name}:")
    for i, (feature, importance) in enumerate(zip(top_features['feature'], top_features['importance'])):
        print(f"{i+1}. {feature}: {importance:.4f}")
else:
    # For models without built-in feature importance, we can use permutation importance
    from sklearn.inspection import permutation_importance
    
    print(f"Computing permutation importance for {best_model_name}...")
    result = permutation_importance(best_model, X_test_selected, y_test, n_repeats=10, random_state=42)
    perm_importance = pd.DataFrame({
        'feature': X_test_selected.columns,
        'importance': result.importances_mean
    }).sort_values('importance', ascending=False)
    
    top_features = perm_importance.head(10)
    print(f"\nTop 10 most important features for {best_model_name} (permutation importance):")
    for i, (feature, importance) in enumerate(zip(top_features['feature'], top_features['importance'])):
        print(f"{i+1}. {feature}: {importance:.4f}")

### Recommendations Based on Feature Importance

Based on the feature importance analysis, we can make the following recommendations:

1. **Trip Distance**: Trip distance is one of the most important predictors of trip duration. This suggests that pricing strategies should be primarily based on distance.

2. **Time of Day**: The hour of pickup is a significant factor. Trips during rush hours take longer, so surge pricing during these times is justified.

3. **Day of Week**: Weekday vs. weekend patterns affect trip duration. Different pricing or resource allocation strategies might be needed for weekdays vs. weekends.

4. **Location**: Pickup and dropoff locations significantly impact trip duration. Certain areas might need more taxis during specific times.

5. **Speed**: Average speed is a key factor. Routes with consistently low speeds might need alternative paths or special pricing.

6. **Payment Method**: If payment method is an important feature, this might indicate different customer behaviors based on how they pay.

### Operational Recommendations:

1. **Dynamic Pricing**: Implement more sophisticated dynamic pricing based on predicted trip duration, not just distance.

2. **Resource Allocation**: Allocate more taxis to areas with high demand and longer predicted trip durations.

3. **Route Optimization**: Identify and optimize routes with consistently longer-than-expected trip durations.

4. **Customer Communication**: Provide more accurate trip duration estimates to customers based on the model's predictions.

5. **Driver Training**: Train drivers on efficient routes for specific pickup/dropoff location pairs that tend to have longer durations.

### Model Improvement Recommendations:

1. **External Data**: Incorporate external data such as weather conditions, events, and traffic patterns.

2. **Temporal Features**: Add more sophisticated temporal features like holidays, events, or seasonal patterns.

3. **Geospatial Features**: Add more detailed geospatial features like distance to major landmarks or traffic hotspots.

4. **Ensemble Methods**: Combine multiple models for better performance.

5. **Real-time Updates**: Implement a system for real-time updates to the model based on current traffic conditions.

## 11. Summary

In this notebook, we have:

1. Loaded the processed NYC taxi data
2. Selected the most important features for modeling
3. Trained a baseline model to establish a performance benchmark
4. Trained and evaluated multiple regression models
5. Compared the performance of all models
6. Analyzed the best-performing model in detail
7. Saved the best model for future use
8. Interpreted the results and made recommendations

The best-performing model was the [Best Model Name], which achieved a test RMSE of [Best RMSE] and a test R² of [Best R²]. This model can be used to predict trip durations for NYC taxi trips with reasonable accuracy.

The most important features for predicting trip duration were [Top Features], which suggests that [Key Insights].

Based on our analysis, we recommend [Key Recommendations].